# 뇌졸중(stroke) 발생 여부

In [2]:
import pandas as pd
import numpy as np

In [3]:
data = pd.read_csv('https://raw.githubusercontent.com/Datamanim/datarepo/main/stroke_/train.csv')
data.head()

,id,gender,age,hypertension,heart_disease,ever_married,work_type,Residence_type,avg_glucose_level,bmi,smoking_status,stroke
0,1192,Female,31,0,0,No,Govt_job,Rural,70.66,27.2,never smoked,0
1,77,Female,13,0,0,No,children,Rural,85.81,18.6,Unknown,0
2,59200,Male,18,0,0,No,Private,Urban,60.56,33.0,never smoked,0
3,24905,Female,65,0,0,Yes,Private,Urban,205.77,46.0,formerly smoked,1
4,24257,Male,4,0,0,No,children,Rural,90.42,16.2,Unknown,0


In [4]:
data.isnull().sum()

id                    0
gender                0
age                   0
hypertension          0
heart_disease         0
ever_married          0
work_type             0
Residence_type        0
avg_glucose_level     0
bmi                  52
smoking_status        0
stroke                0
dtype: int64

In [5]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1137 entries, 0 to 1136
Data columns (total 12 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   id                 1137 non-null   int64  
 1   gender             1137 non-null   object 
 2   age                1137 non-null   object 
 3   hypertension       1137 non-null   int64  
 4   heart_disease      1137 non-null   int64  
 5   ever_married       1137 non-null   object 
 6   work_type          1137 non-null   object 
 7   Residence_type     1137 non-null   object 
 8   avg_glucose_level  1137 non-null   float64
 9   bmi                1085 non-null   float64
 10  smoking_status     1137 non-null   object 
 11  stroke             1137 non-null   int64  
dtypes: float64(2), int64(4), object(6)
memory usage: 106.7+ KB


Q1. 성별이 Male인 환자들의 age의 평균값은 ?

In [6]:
data[data.age.str.startswith('*')]

,id,gender,age,hypertension,heart_disease,ever_married,work_type,Residence_type,avg_glucose_level,bmi,smoking_status,stroke
586,12363,Male,*82,0,1,Yes,Govt_job,Urban,74.1,28.8,Unknown,1


In [7]:
data[data.age.str.startswith('*')].age.replace('*82', 82)

586    82
Name: age, dtype: int64

In [8]:
data.loc[data.age.str.startswith('*'), 'age'] = 82

In [9]:
data.age = data.copy().age.astype(int)

In [10]:
data[data['gender']=='Male']['age'].mean()

44.68623481781376

Q2. bmi컬럼의 결측치를 bmi컬럼의 결측치를 제외한 나머지 값들의 중앙값으로 채웠을 경우 bmi 컬럼의 평균을 소숫점 이하 3자리 까지 구하여라

In [11]:
round(data.bmi.fillna(data.bmi.median()).mean(), 3)

29.166

Q3. bmi컬럼의 각 결측치들을 직전의 행의 bmi값으로 채웠을 경우 bmi 컬럼의 평균을 소숫점 이하 3자리 까지 구하여라

In [12]:
round(data.bmi.fillna(method='ffill').mean(), 3)

29.188

Q4. bmi컬럼의 각 결측치들을 결측치를 가진 환자 나이대(10단위)의 평균 bmi 값으로 대체한 후 대체된 bmi 컬럼의 평균을 소숫점 이하 3자리 까지 구하여라

In [18]:
data['age_group'] = (data['age']/10).astype(int)*10
data.head()

,id,gender,age,hypertension,heart_disease,ever_married,work_type,Residence_type,avg_glucose_level,bmi,smoking_status,stroke,age_group
0,1192,Female,31,0,0,No,Govt_job,Rural,70.66,27.2,never smoked,0,30
1,77,Female,13,0,0,No,children,Rural,85.81,18.6,Unknown,0,10
2,59200,Male,18,0,0,No,Private,Urban,60.56,33.0,never smoked,0,10
3,24905,Female,65,0,0,Yes,Private,Urban,205.77,46.0,formerly smoked,1,60
4,24257,Male,4,0,0,No,children,Rural,90.42,16.2,Unknown,0,0


In [20]:
set(data['age_group'].unique())

{0, 10, 20, 30, 40, 50, 60, 70, 80}

In [22]:
bmi_mean = {}
for i in set(data['age_group'].unique()):
    bmi_mean[i] = data[(data['age_group']==i) & (data['bmi'].notnull())]['bmi'].mean()
bmi_mean

{0: 18.960784313725487,
 70: 30.447727272727274,
 40: 31.427096774193554,
 10: 24.47722772277227,
 80: 28.566666666666666,
 50: 31.894086021505373,
 20: 28.05274725274725,
 60: 31.615873015873017,
 30: 31.751111111111108}

In [38]:
data[data.bmi.isnull()==True].index

Int64Index([], dtype='int64')

In [36]:
for index in data[data.bmi.isnull()==True].index:
    data.loc[index, 'bmi'] = bmi_mean[data.iloc[index]['age_group']]

In [39]:
round(data.bmi.mean(), 3)

29.263

In [40]:
# 답안 예시

# import pandas as pd
# df = pd.read_csv('https://raw.githubusercontent.com/Datamanim/datarepo/main/stroke_/train.csv')
# # Noise 데이터 int형 변환
# df['age'] = df['age'].str.replace('*','').astype('int')

# # 결측치 제외 나이대별 평균값 계산 및 dictionary 형태로 변환
# mean = df[df.bmi.notnull()].groupby(df.age//10 *10).bmi.mean()
# dic = { x:y for x,y in mean.items()}


# idx =df.loc[df.bmi.isnull(),['age','bmi']].index
# # 결측치 매핑
# df.loc[df.bmi.isnull(),'bmi']  =(df[df.bmi.isnull()].age//10*10).map(lambda x : dic[x])

# result = df.bmi.mean()
# print(result)

Q5. avg_glucose_level 컬럼의 값이 200이상인 데이터를 모두 199로 변경하고 stroke값이 1인 데이터의 avg_glucose_level값의 평균을 소수점이하 3자리 까지 구하여라

In [43]:
data.loc[data[data['avg_glucose_level']>=200].index, 'avg_glucose_level'] = 199

In [46]:
round(data[data.stroke==1]['avg_glucose_level'].mean(), 3)

125.188